In [ ]:
#
# Notebook: 4_xai_examples.ipynb
#

import ibbi

# --- 1. Setup ---
# Let's use the multi-class species classifier for our explanations.
model = ibbi.create_model("species_classifier", pretrained=True)
explainer = ibbi.Explainer(model)

# Load the test dataset and select an image to explain.
print("Loading the test dataset...")
test_dataset = ibbi.get_dataset()
image_to_explain = test_dataset[1010]["image"]  # Use an image from the test dataset
print("Dataset loaded and an image has been selected for explanation.")


# --- 2. Explainability with LIME ---
# LIME (Local Interpretable Model-agnostic Explanations) is great for a quick
# visual explanation of which parts of the image contributed to a prediction.
print("\n--- Generating LIME Explanation ---")
# This returns the explanation object and the original image.
lime_explanation, original_image = explainer.with_lime(image_to_explain)

# You can then plot the explanation.
# This will show the original image and then an overlay highlighting important regions.
ibbi.plot_lime_explanation(lime_explanation, original_image, top_k=1)


# --- 3. Explainability with SHAP ---
# SHAP (SHapley Additive exPlanations) is more computationally intensive but provides
# more robust, theoretically grounded explanations.
print("\n--- Generating SHAP Explanation ---")

# SHAP requires a background dataset to represent the "absence" of features.
# `ibbi` provides a default one you can load.
background_dataset = ibbi.get_shap_background_dataset()

# We also need a small dataset of images to explain. For this example,
# we'll just use our single image from the test dataset.
explain_dataset = [{"image": image_to_explain}]

# Generate the SHAP values. This is the most time-consuming step.
# `max_evals` controls the number of model evaluations. Higher is more accurate but slower.
shap_values = explainer.with_shap(
    explain_dataset=explain_dataset,
    background_dataset=background_dataset,
    num_explain_samples=1,  # We are only explaining one image
    max_evals=500,
)

# Plot the explanation for the first (and only) image in our set.
# This will generate plots for the top predicted classes.
ibbi.plot_shap_explanation(shap_values[0], model)